In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import stats
import seaborn as sns

In [ ]:
df=pd.read_csv('development.csv')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sum()

In [ ]:
#make a new column of counts of unique host_ids
adf=df.host_id.value_counts().reset_index(name='countIn')
adf=adf.rename(columns={'index': 'host_id'})
df=pd.merge(df, adf, on='host_id')

In [ ]:
#replace null values with 0
df=df.fillna(0)

In [ ]:
#selecting relevant columns
df=df[[
       'room_type','price',
       'minimum_nights', 
        'calculated_host_listings_count',
       'availability_365','number_of_reviews','countIn']]

In [ ]:
#deleting rows with price =0
df=(df[df['price']>0])


In [ ]:
#drop duplicates if any
df.drop_duplicates

In [ ]:
plt.figure(figsize=(5,5))
ax = sns.histplot(x="number_of_reviews",data=df,binwidth=20, stat="probability",cumulative=True)
ax.set_xlim(0,df['number_of_reviews'].max())
ax.set_title("Dist. of price")

In [ ]:
plt.figure(figsize=(5,5))

ax = sns.histplot(x="availability_365",data=df,binwidth=20, stat="probability",cumulative=True)
ax.set_xlim(0,500)
ax.set_title("Dist. availability_365")

In [ ]:
#Removing outliers from the dataset
df= df[(df["minimum_nights"] < 30) & (df["price"] < 500) & (df["availability_365"] < 365)& (df["number_of_reviews"] < 100) ]

In [ ]:

#One hot encoding for rootype column
one_hot = pd.get_dummies(df['room_type'])
df = df.drop('room_type',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df

In [ ]:
df

In [ ]:
df.reset_index(drop='inplace')


In [ ]:
predictors=df.columns.tolist()
predictors.remove('price')
print(len(predictors))

8


In [ ]:
from sklearn.model_selection import train_test_split
target_column = ['price']

X = df[predictors].values
y = df[target_column].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=2)
print(X_train.shape); print(X_test.shape)

(25247, 8)
(6312, 8)


In [ ]:
#linear regression
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)
rmse_train = np.sqrt(mean_squared_error(y_test,y_pred))
r2_train = r2_score(y_test,y_pred)
print('Residual Error (MSE):',rmse_train)
print('Mean Absolute error:',r2_train)


Residual Error (MSE): 68.28661443478246
Mean Absolute error: 0.34098554646485413


In [ ]:


#Ridge regressions
rr = Ridge(alpha=1.0)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print(r2_score(y_train, pred_train_rr))

pred_test_rr= rr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print(r2_score(y_test, pred_test_rr))


68.44283748484283
0.3489341527325841
68.28649897412343
0.3409877750181193
